In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

import re
import requests
from bs4 import BeautifulSoup

import time
from pathlib import Path

# GetDate
from datetime import datetime #,  date

import glob
import os.path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



In [2]:
#Reimportando arquivo - caso execucao tenha sido pausada
#Identificacao de arquivo mais recente
folder_path = r'./ARQ/'
file_type = r'/*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

path_to_file = max_file.replace('\\','/')

#Em caso de definicao manual usar esta linha
#path_to_file = './ARQ/Listagem_13-22.csv'

listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

In [4]:
listagem['Autor'].unique().tolist()

['Oliveira Tresse',
 'Júlio Gasparette',
 'Luiz Otávio Fernandes Coelho - Pardal',
 'Cido',
 'Noraldino Júnior',
 'Zé Márcio',
 'Wanderson Castelar',
 'Jucelio',
 'Betão',
 'Vagner de Oliveira',
 'Dr. Fiorilo',
 'Ana Rossignoli',
 'Chico Evangelista',
 'André Mariano',
 'Dr. Fiorilo, Luiz Otávio Fernandes Coelho - Pardal, Wanderson Castelar',
 'Cido Reis',
 'Isauro Calais',
 'Dr. Antônio Aguiar',
 'Nilton Militão',
 'Rodrigo Mattos',
 'Ana Rossignoli, Dr. Antônio Aguiar, Luiz Otávio Fernandes Coelho - Pardal, Betão',
 'Ana Rossignoli, André Mariano, Antônio Almas, Dr. Antônio Aguiar, Cido Reis, Chico Evangelista, Vagner de Oliveira, Isauro Calais, Dr. Fiorilo, Zé Márcio, João do Joaninho, Jucelio, Júlio Gasparette, Luiz Otávio Fernandes Coelho - Pardal, Nilton Militão, Noraldino Júnior, Oliveira Tresse, Betão, Rodrigo Mattos, Wanderson Castelar',
 'Isauro Calais, Ana Rossignoli, André Mariano, Dr. Antônio Aguiar, Cido Reis, Chico Evangelista, Vagner de Oliveira, Dr. Fiorilo, Zé Márcio,

In [18]:
list_sem_multiautor = listagem[~listagem['Autor'].str.contains(",")]
list_sem_multiautor['Autor'].unique().tolist()

['Oliveira Tresse',
 'Júlio Gasparette',
 'Luiz Otávio Fernandes Coelho - Pardal',
 'Cido',
 'Noraldino Júnior',
 'Zé Márcio',
 'Wanderson Castelar',
 'Jucelio',
 'Betão',
 'Vagner de Oliveira',
 'Dr. Fiorilo',
 'Ana Rossignoli',
 'Chico Evangelista',
 'André Mariano',
 'Cido Reis',
 'Isauro Calais',
 'Dr. Antônio Aguiar',
 'Nilton Militão',
 'Rodrigo Mattos',
 'João do Joaninho',
 'Executivo',
 'José Laerte',
 'José Emanuel',
 'Leo de Oliveira',
 'Mesa Diretora - biênio 2017/2018',
 'Delegada Sheila',
 'Kennedy Ribeiro',
 'Sargento Mello Casal',
 'Dr. Adriano Miranda',
 'Charlles Evangelista',
 'Marlon Siqueira',
 'João Coteca',
 'Júlio Obama Jr.',
 'Wagner do Sindicato',
 'Juraci Scheffer',
 'Mesa Diretora - Biênio 2019/2020',
 'João Wagner',
 'André Luiz',
 'Tiago Bonecão',
 'Protetora Kátia Franco',
 'Maurício Delgado',
 'Tallia Sobral',
 'Cida Oliveira',
 'Laiz Perrut',
 'Bejani Júnior',
 'Julinho Rossignoli']

In [3]:
listagem

,Projeto,Ano,Tipo,Ementa,Situacao,Autor,Codigo_Tipo,Id_Projeto_Camara,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,40520,2013,Requerimento,"Solicita urbanização na rua Antônio Celeste, n...",Arquivada,Oliveira Tresse,REQ,44205,11/07/2013,SIM,SIM
1,4028,2013,Requerimento,Solicita limpeza nas bocas de lobo no bairro C...,Arquivada,Júlio Gasparette,REQ,44107,09/07/2013,SIM,SIM
2,36840,2013,Requerimento,solicita recapemamento asfáltico na rua Orland...,Em Tramitação,Luiz Otávio Fernandes Coelho - Pardal,REQ,43978,31/12/2000,NAO,Informacao nao encontrada
3,891,2013,Requerimento,Solicita o asfaltamento de toidas as ruas da C...,Arquivada,Cido,REQ,31106,05/02/2013,SIM,SIM
4,8890,2013,Requerimento,Solicita o asfaltamento de todas as ruas da co...,Arquivada,Cido,REQ,27368,05/02/2013,SIM,SIM
...,...,...,...,...,...,...,...,...,...,...,...
74093,1,2022,Emenda,"Substitui os artigos 2º e 4ª, do Projeto de Le...",Aprovado,Nilton Militão,EMENDA,-1126,,,
74094,1,2022,Emenda,Acrescenta o art.6º ao Projeto de Lei oriundo ...,Aguardando leitura em plenário,"André Luiz, Dr. Antônio Aguiar",EMENDA,-1127,,,
74095,1,2022,Emenda,"Substitui o caput do artigo 1º, o inciso I do ...",Aprovado,"Luiz Otávio Fernandes Coelho - Pardal, Cida Ol...",EMENDA,-1128,,,
74096,1,2022,Projeto Substitutivo,"“Altera dispositivos da Lei Nº 10.777, de 15 d...",Aprovado,"Zé Márcio, Bejani Júnior, Sargento Mello Casal...",PSUB,121457,13/12/2022,SIM,SIM
